In [17]:
from scipy.stats import norm
import numpy as np
import yfinance as yf
import pandas as pd

from datetime import datetime


def get_historical_data(stk, start_date, end_date):
    df = yf.download(stk, start_date, end_date, interval="1d").dropna()
    return df


def group_by_time_interval(df, number_of_weeks_in_group):
    df["start"] = df.index
    df["end"] = df.index
    df["day_of_week"] = df.index.dayofweek
    df["week_of_year"] = df.index.isocalendar().week + df.index.isocalendar().year * 100
    df["month_of_year"] = df.index.strftime("%m-%Y")
    df["week_group"] = (df.week_of_year / number_of_weeks_in_group).apply(np.ceil)
    df = df.groupby(["week_group"]).agg(
        {"start": "first", "end": "last", "Open": "first", "Close": "last", "Volume":"sum"}
    )
    #df['volume_rank']=df.Volume.rank(pct=True).shift(1)

    df["days"] = (df["end"] - df["start"]).dt.days + 1
    df = df.set_index("start")
    return df


def trade_parameters(df, yield_perc):
    df["strike_price"] = df.Open.round(0)
    df["straddle_yield"] = yield_perc
    df["straddle_premium"] = df.strike_price * yield_perc
    df["price_change_abs"] = abs(df.Close - df.strike_price)
    df["profit_loss"] = df.straddle_premium - df.price_change_abs
    df["log_return"] = np.log((df.Open + df.profit_loss) / df.Open)
    

    df['volume_rank']=df.Volume.rank(pct=True)
    
    df['volume_change_pct']=(df.Volume-df.Volume.shift(1))/df.Volume.shift(1)
    
    df['abs_change_pct']=(abs((df.Close - df.Open)/df.Open))
    
    #df['abs_change_prev_roll_2s']=df.abs_change.rolling(2).sum().shift(1)
    #df['abs_change_prev']=df.abs_change.shift(1)
    
   # df['abs_change_rank']=df.abs_change.rank(pct=True)
    
    #df['change_rank_prev']=df.change_rank.shift(1)
    
    
    #df=df[df.volume_rank1<0.20]
    #df=df[df.change_rank1<0.20]
    return df


atm_option_data = pd.read_csv("option_data_2022-08-19_2022-08-11.csv")


start_date = "2015-12-01"
end_date = "2022-08-11"
number_of_weeks_in_group = 2
arr = []
for i in range(0, len(atm_option_data.symbol)):
    stock = atm_option_data.symbol[i]
    yield_perc = (atm_option_data.atm_call_last[i] + atm_option_data.atm_put_last[i]) / atm_option_data.strike[i]
    df = get_historical_data(stock, start_date, end_date)
    df = group_by_time_interval(df, number_of_weeks_in_group)
    df = trade_parameters(df, yield_perc)
    df.to_csv(stock + ".csv")
    log_r = np.sum(df.log_return)
    pos_log = np.sum(df[df.log_return > 0].log_return)
    neg_log = np.sum(df[df.log_return <= 0].log_return)
    pos_periods = len(df[df.log_return > 0].log_return)
    ttl_periods = len(df.log_return)
    avg_yield = np.average(df.log_return.dropna())
   # vol_rank=df.volume_rank.iloc[-1]
    #lr_rank=df.change_rank.iloc[-1]
    
    
    cor_1=np.corrcoef(df.abs_change_pct.iloc[2:],df.abs_change_pct.shift(1).iloc[2:])[0,1]
    cor_2=np.corrcoef(df.abs_change_pct.iloc[2:], df.Volume.shift(1).iloc[2:])[0,1]
    cor_3=np.corrcoef(df.abs_change_pct.iloc[2:],df.abs_change_pct.rolling(2).sum().shift(1).iloc[2:])[0,1]
    cor_4=np.corrcoef(df.abs_change_pct.iloc[2:],
                      df.abs_change_pct.shift(1).iloc[2:] 
                      * df.Volume.shift(1).iloc[2:])[0,1]
    cor_5=np.corrcoef(df.abs_change_pct.iloc[2:],
                      (df.abs_change_pct.shift(1).iloc[2:] * 
                      df.abs_change_pct.rolling(2).sum().shift(1).iloc[2:])/2)[0,1]
    arr.append(
        [
            stock,
            atm_option_data.last_price[i],
            atm_option_data.atm_call_last[i],
            atm_option_data.atm_put_last[i],
            yield_perc,
            log_r,
            pos_log,
            neg_log,
            pos_periods,
            ttl_periods,
            avg_yield,
            cor_1,
            cor_2,
            cor_3,
            cor_4,
            cor_5
        ]
    )

finaldf = pd.DataFrame(
    data=arr,
    columns=[
        "stock",
        "last_price",
        "atm_call_price",
        "atm_put_price",
        "straddle_yield",
        "log_r",
        "pos_log",
        "neg_log",
        "pos_periods",
        "ttl_periods",
        "avg_yield",
        'cor_1',
        'cor_2',
        'cor_3',
        'cor_4',
        'cor_5',
    ],
)
finaldf["sharpe"] = abs(finaldf.pos_log / finaldf.neg_log)
finaldf["winr"] = finaldf.pos_periods / finaldf.ttl_periods
finaldf.sort_values(by="log_r", ascending=False, inplace=True)
finaldf.to_csv("big.csv")
print(np.average(finaldf.cor_1.dropna()))
print(np.average(finaldf.cor_2.dropna()))
print(np.average(finaldf.cor_3.dropna()))
print(np.average(finaldf.cor_4.dropna()))
print(np.average(finaldf.cor_5.dropna()))

[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%********